# Installing the OpenBB Platform in Google Colab

This notebook will install the OpenBB Platform, fetch some data and prepare it for display as a bar chart.

Sign up for a free account here: https://my.openbb.co

In [4]:
# Install the OpenBB Platform with all available extensions.
# Messages indicating package version conflicts at the end of installation can be safely ignored.

!pip install openbb[all]

# There is also a nightly distribution available, openbb-nightly

In [5]:
# Before running this cell, restart the runtime by selecting, "Restart runtime", from the "Runtime" menu.

# Import statements - for many scenarios, the only import needed will be `from openbb import obb`
from typing import Literal
from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets
import pandas as pd
import pandas_ta as ta
from datetime import datetime
from plotly import graph_objects as go

from openbb import obb

In [7]:
# Login to OpenBB Hub to retrieve stored API keys.
# https://my.openbb.co/app/platform/pat
# https://my.openbb.co/app/platform/api-keys

obb.account.login(pat="replace with your PAT")

# This is not required

AttributeError: 'App' object has no attribute 'account'

In [ ]:
# Verify that the credentials from Hub were loaded successfully.

obb.user.credentials

In [ ]:
# Set the output preference, if desired. The examples below use Pandas DataFrames.

obb.user.preferences.output_type = "dataframe"

In [ ]:
# Get Some Data
symbol = "SPY"

options = obb.derivatives.options.chains(symbol, provider="cboe")

options

In [ ]:
# Prepare A View - Volume and Open Interest by Expiration or Strike

def filter_options_data(options, by: Literal["expiration", "strike"] = "expiration"):
    data = pd.DataFrame()
    data["Total Open Interest"] = options.groupby(by)["open_interest"].sum()
    data["Call Open Interest"] = options[options["option_type"] == "call"].groupby(by)["open_interest"].sum()
    data["Put Open Interest"] = options[options["option_type"] == "put"].groupby(by)["open_interest"].sum()
    data["Total Volume"] = options.groupby(by)["volume"].sum()
    data["Call Volume"] = options[options["option_type"] == "call"].groupby(by)["volume"].sum()
    data["Put Volume"] = options[options["option_type"] == "put"].groupby(by)["volume"].sum()

    return data

data = filter_options_data(options, "strike")

data

In [ ]:
# Do not run this cell if you are following the example above.

# In this scenario, "data" could be anything that would be displayed as a bar chart. Alternatively, it could be company fundamentals data.

# Note: This requires a valid FMP API key


symbol="AAPL"

data = obb.equity.fundamental.ratios(symbol, limit = 100, period="quarter", provider="fmp")

data.index = data.index.strftime("%Y-%m-%d")

data

In [ ]:
# Create a widget for selecting the data to display.

clear_output(wait = False)

data_choices = data.columns.tolist()
data_selection = widgets.Dropdown(
    options = data_choices,
    value = None,
)
output = widgets.Output()


def generate_figure(data, data_choice):
    data = data[data[data_choice].notnull()]
    fig = go.Figure()
    fig.add_bar(
        y = data[data_choice][data[data_choice] > 0].values,
        x = data[data_choice][data[data_choice] > 0].index,
        name = data_choice,
        marker = dict(color = "blue"),
    )
    fig.add_bar(
        y = data[data_choice][data[data_choice] < 0].values,
        x = data[data_choice][data[data_choice] < 0].index,
        name = data_choice,
        marker = dict(color = "red")
    )
    fig.update_xaxes(type="category")
    fig.update_traces(width=0.98, selector=dict(type="bar"))
    fig.update_layout(
        showlegend=False,
        width=1400,
        height=600,
        title = dict(
            text=f"{symbol} {data_choice.replace('_', ' ').title()}",
            xanchor = "center",
            x = 0.5,
            font = dict(size = 20)
        ),
        barmode="overlay",
        bargap=0,
        bargroupgap=0,
        yaxis=dict(
            ticklen=0,
            showgrid=True,
            tickfont=dict(size=14),
        ),
        xaxis=dict(
            showgrid=False,
            autorange=True,
            tickangle=90,
            tickfont=dict(size=11),
        ),
    )
    return fig

def on_value_change(change):
    clear_output(wait = True)
    display(data_selection)
    with output:
        data_selection.value

data_selection.observe(on_value_change, names="value")
display(data_selection)

# Select from the drop-down menu below.

In [ ]:
# Play this cell to display the choice

if data_selection.value is not None:

    generate_figure(data, data_selection.value).show()